# Publish Event For A Model In Fiddler
For this section, we will cover how to publish an event for a model in Fiddler.
We will do so by sending monitoring events for a specific model and 
project that is already housed in Fiddler.

## Initialize Fiddler Client
We begin this section as usual by establishing a connection to our
Fiddler instance. We can establish this connection either by specifying 
our credentials directly, or by utilizing our `fiddler.ini` file. More
information can be found in the [setup](https://github.com/fiddler-labs/fiddler-samples/blob/master/content_root/tutorial/00%20Setup.ipynb) section.

In [ ]:
import fiddler as fdl

# client = fdl.FiddlerApi(url=url, org_id=org_id, auth_token=auth_token)
client = fdl.FiddlerApi()

## Load event log
Event logs must contains the model's input features and predictions. For this 
demonstration, we have collected events and saved them in a file called `events.log`.

In [ ]:
import pandas as pd
event_log = pd.read_csv('/app/fiddler_samples/samples/datasets/winequality/events.log')

project_id = 'tutorial'
model_id = 'wine_quality_model'

## Publish Events
In this step, we will be simulating traffic to send for our model monitoring by using 
[publish_event](https://docs.fiddler.ai/api-reference/python-package/#publish-event). 
This will be the equivalent of running our model separately on data, and either 
sending to Fiddler then, or saving this information to a log and sending at a later point.

For this demonstration, we will be going with a log-related approach. 
This log contains rows that have inputs and predictions. 
To most accurately simulate this as a time-series event, we will also be calling 
a function to generate a timestamp in the last two weeks. Real data will ideally 
have a timestamp related to when the event took place; otherwise, the current 
time will be used.

**Note**: The timestamp must be in UTC milliseconds. See 
[here](https://docs.fiddler.ai/api-reference/python-package/#publish-event) for more details

In [ ]:
import datetime
import time
from IPython.display import clear_output
from random import sample, randint
NUM_EVENTS_TO_SEND = 50

def getTimestampFromPastTwoWeeks():
    """
    Generate a randomized timestamp from the past two weeks. Timestamp is in 
    milliseconds since epoch in UTC.
    """
    TWO_WEEKS_MS = 604800 * 2 * 1000
    current_time_in_ms = round(time.time() * 1000)
    
    random_time_in_past_two_weeks = current_time_in_ms - randint(0, TWO_WEEKS_MS)
    return random_time_in_past_two_weeks

# Convert this dataframe into a list of dictionary events, where each event is its own dictionary
event_list_dict = event_log.sample(n=NUM_EVENTS_TO_SEND, random_state=42).to_dict(orient='records') 

for ind, event_dict in enumerate(event_list_dict):
    event_ms_time_stamp = getTimestampFromPastTwoWeeks()
    result = client.publish_event(project_id, model_id, event_dict, event_time_stamp=event_ms_time_stamp)
    
    clear_output(wait = True)
    readable_timestamp = datetime.datetime.fromtimestamp(event_ms_time_stamp/1000.0)
    
    print(f'Sending {ind+1} / {NUM_EVENTS_TO_SEND} \n{readable_timestamp} UTC: \n{event_dict}')
    time.sleep(1)